# Collision-Free Trajectory with Base Movement

This notebook demonstrates collision-free motion planning where the robot's base (position and orientation) is also part of the motion plan.

The robot will move its base AND arm to avoid obstacles!

In [ ]:
import time

import numpy as np

import skrobot
from skrobot.model.primitives import Axis
from skrobot.model.primitives import Box
from skrobot.model.primitives import LineString
from skrobot.planner import sqp_plan_trajectory
from skrobot.planner import SweptSphereSdfCollisionChecker
from skrobot.planner.utils import get_robot_config
from skrobot.planner.utils import set_robot_config

## Setup Robot and Obstacle

In [ ]:
# Initialize robot and obstacle
np.random.seed(0)
robot_model = skrobot.models.PR2()
robot_model.init_pose()

# Create obstacle
box_center = np.array([0.9, -0.2, 0.9])
box = Box(extents=[0.7, 0.5, 0.6], with_sdf=True)
box.translate(box_center)

# Define robot links
link_list = [
    robot_model.r_shoulder_pan_link, robot_model.r_shoulder_lift_link,
    robot_model.r_upper_arm_roll_link, robot_model.r_elbow_flex_link,
    robot_model.r_forearm_roll_link, robot_model.r_wrist_flex_link,
    robot_model.r_wrist_roll_link]
joint_list = [link.joint for link in link_list]

coll_link_list = [
    robot_model.r_upper_arm_link, robot_model.r_forearm_link,
    robot_model.r_gripper_palm_link, robot_model.r_gripper_r_finger_link,
    robot_model.r_gripper_l_finger_link]

print("Robot and obstacle created")

## Setup Start and Goal Configurations

**Important**: With base movement enabled, the configuration includes:
- 7 joint angles for the right arm
- 3 base parameters: [x, y, theta]

In [ ]:
# Start configuration: arm angles + base pose
av_start = np.array([0.564, 0.35, -0.74, -0.7, -0.7, -0.17, -0.63])
base_pose_start = [-0.5, 0.8, 0]  # [x, y, theta]
av_start = np.hstack([av_start, base_pose_start])

print(f"Start config (7 joints + 3 base): {av_start}")
print(f"  Joints: {av_start[:7]}")
print(f"  Base [x, y, theta]: {av_start[7:]}")

In [ ]:
# Solve IK for goal configuration
joint_angles = np.deg2rad([-60, 74, -70, -120, -20, -30, 180])
set_robot_config(robot_model, joint_list, joint_angles)
target_coords = skrobot.coordinates.Coordinates([0.8, -0.6, 0.8], [0, 0, 0])

right_arm_end_coords = skrobot.coordinates.CascadedCoords(
    parent=robot_model.r_gripper_tool_frame,
    name='right_arm_end_coords')
robot_model.inverse_kinematics(
    target_coords=target_coords,
    link_list=link_list,
    move_target=robot_model.right_arm_end_coords, rotation_axis=True)
av_goal = get_robot_config(robot_model, joint_list, with_base=True)

print(f"Goal config (7 joints + 3 base): {av_goal}")
print(f"  Joints: {av_goal[:7]}")
print(f"  Base [x, y, theta]: {av_goal[7:]}")

## Setup Collision Checker and Viewer

In [ ]:
# Setup collision checker
sscc = SweptSphereSdfCollisionChecker(box.sdf, robot_model)
for link in coll_link_list:
    sscc.add_collision_link(link)

print("Collision checker configured")

In [ ]:
# Create viewer and add objects
viewer = skrobot.viewers.JupyterNotebookViewer(height=600)
viewer.add(robot_model)
viewer.add(box)
viewer.add(Axis(pos=target_coords.worldpos(), rot=target_coords.worldrot()))

sscc.add_coll_spheres_to_viewer(viewer)

# Set start configuration (don't show yet - will show in animation cell)
set_robot_config(robot_model, joint_list, av_start, with_base=True)
viewer.set_camera([0, 0, np.pi / 2.0])

print("Viewer configured (will display in animation cell below)")

## Plan Trajectory

The planner will find a collision-free path that moves both the base and arm.

In [ ]:
print("Planning trajectory with base movement...")
n_waypoint = 10
av_seq = sqp_plan_trajectory(
    sscc, av_start, av_goal, joint_list, n_waypoint,
    safety_margin=5.0e-2, with_base=True)
print(f"✓ Generated {len(av_seq)} waypoints")
print(f"  Each waypoint has {len(av_seq[0])} values (7 joints + 3 base params)")

## Animate Trajectory

Watch the robot move its base AND arm to avoid the obstacle!

Notice:
- Base position (x, y) changes
- Base orientation (theta) changes  
- Arm joints change
- All updates are smooth without flickering!

In [ ]:
print("Animating trajectory...")

# Display viewer in THIS cell
viewer.show()

arm_point_history = []
line_string = None

for i, av in enumerate(av_seq):
    # Update robot configuration (joints + base)
    set_robot_config(robot_model, joint_list, av, with_base=True)
    arm_point_history.append(right_arm_end_coords.worldpos())

    # Update trajectory visualization
    if line_string is not None:
        viewer.delete(line_string)
    if len(arm_point_history) > 1:
        line_string = LineString(np.array(arm_point_history))
        viewer.add(line_string)

    # Update collision sphere colors
    sscc.update_color()

    # Update display
    viewer.redraw()

    # Print waypoint info
    joints = av[:7]
    base = av[7:]
    print(f"Waypoint {i + 1}/{len(av_seq)}: base=[{base[0]:.2f}, {base[1]:.2f}, {np.rad2deg(base[2]):.1f}°]")

    time.sleep(1.0)

print("\n✓ Animation complete!")

## Save as HTML

In [ ]:
html = viewer.to_html()
output_file = '/tmp/collision_free_with_base.html'
with open(output_file, 'w') as f:
    f.write(html)
print(f"Saved to: {output_file}")
print("Open this file in a browser to view the final state!")